In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict

from joblib import Parallel,delayed

import pandas as pd

from pathlib import Path

In [2]:
columns = ["key","HL","Release","Press"]
feature_columns = ["key","HL","IL","PL","RL","CL"]
columns_idx = {col:i for i,col in enumerate(columns)}
columns_idx.update({col:i for i,col in enumerate(feature_columns)})

In [3]:
def extract_features(sample):
    og_sample = sample
    sample = sample.values
    features = np.empty((len(sample),len(feature_columns)),dtype=object)
    
    features[:,columns_idx["HL"]] = sample[:,columns_idx["HL"]]
    features[:,columns_idx["key"]] = sample[:,columns_idx["key"]]

    features[1:,columns_idx["IL"]] = sample[1:,columns_idx["Press"]] - sample[:-1,columns_idx["Release"]]
    features[1:,columns_idx["PL"]] = sample[1:,columns_idx["Press"]] - sample[:-1,columns_idx["Press"]]
    features[1:,columns_idx["RL"]] = sample[1:,columns_idx["Release"]] - sample[:-1,columns_idx["Release"]]
    features[1:,columns_idx["CL"]] = sample[1:,columns_idx["Release"]] - sample[:-1,columns_idx["Press"]]

    features = pd.DataFrame(data=features,columns=feature_columns)
    features = pd.concat([features,og_sample.drop(["HL","key"],axis=1)],axis=1)
    
    return features

In [4]:
def pipeline(sample_path,save_dir):
    sample = pd.read_csv(sample_path,names=columns,header=None)
    features = extract_features(sample)
    features.to_csv(save_dir/sample_path.name,index=False)

In [5]:
save_dir = Path("features")
save_dir.mkdir(exist_ok=True)

data_path1 = Path("neuroQWERTY/MIT-CS1PD/data_MIT-CS1PD")
save_dir1 = save_dir/"MIT-CS1PD"
save_dir1.mkdir(exist_ok=True)

data_path2 = Path("neuroQWERTY/MIT-CS2PD/data_MIT-CS2PD")
save_dir2 = save_dir/"MIT-CS2PD"
save_dir2.mkdir(exist_ok=True)

In [6]:
Parallel(n_jobs=-2)(
    delayed(pipeline)(sample_path,save_dir1) for sample_path in data_path1.glob("*.csv")
);

In [7]:
Parallel(n_jobs=-2)(
    delayed(pipeline)(sample_path,save_dir2) for sample_path in data_path2.glob("*.csv")
);